<a href="https://colab.research.google.com/github/nsydn/ise301_modopt2/blob/main/ise301_ip_lockbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install pulp
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 5.2 MB/s 


In [2]:
regions = ['west','midwest','east','south']
cities = ['city1','city2','city3','city4']

In [3]:
import pandas as pd

In [4]:
delays = pd.read_excel('lockbox_data.xlsx','delays',index_col=0)
amounts = pd.read_excel('lockbox_data.xlsx','amounts',index_col=0)

In [10]:
cost = pd.DataFrame(delays.values * amounts.values * 0.2 / 1000, columns=delays.columns, index=delays.index)

In [11]:
cost

,city1,city2,city3,city4
delay,,,,
west,28.0,84.0,112.0,112.0
midwest,60.0,20.0,50.0,50.0
east,96.0,60.0,24.0,60.0
south,64.0,40.0,40.0,16.0


In [12]:
model = LpProblem('Lockbox_v1',LpMinimize)

In [17]:
x = LpVariable.dicts('x',[(i,j) for i in regions for j in cities],cat='Binary')
y = LpVariable.dicts('y',[j for j in cities],cat='Binary')

In [21]:
z1 = lpSum([cost.loc[i,j]*x[i,j] for i in regions for j in cities])
z2 = lpSum([50*y[j] for j in cities])

In [22]:
model += z1 + z2

In [23]:
model

Lockbox_v1:
MINIMIZE
96.0*x_('east',_'city1') + 60.0*x_('east',_'city2') + 24.0*x_('east',_'city3') + 60.0*x_('east',_'city4') + 60.0*x_('midwest',_'city1') + 20.0*x_('midwest',_'city2') + 50.0*x_('midwest',_'city3') + 50.0*x_('midwest',_'city4') + 64.0*x_('south',_'city1') + 40.0*x_('south',_'city2') + 40.0*x_('south',_'city3') + 16.0*x_('south',_'city4') + 28.0*x_('west',_'city1') + 84.0*x_('west',_'city2') + 112.0*x_('west',_'city3') + 112.0*x_('west',_'city4') + 50*y_city1 + 50*y_city2 + 50*y_city3 + 50*y_city4 + 0.0
VARIABLES
0 <= x_('east',_'city1') <= 1 Integer
0 <= x_('east',_'city2') <= 1 Integer
0 <= x_('east',_'city3') <= 1 Integer
0 <= x_('east',_'city4') <= 1 Integer
0 <= x_('midwest',_'city1') <= 1 Integer
0 <= x_('midwest',_'city2') <= 1 Integer
0 <= x_('midwest',_'city3') <= 1 Integer
0 <= x_('midwest',_'city4') <= 1 Integer
0 <= x_('south',_'city1') <= 1 Integer
0 <= x_('south',_'city2') <= 1 Integer
0 <= x_('south',_'city3') <= 1 Integer
0 <= x_('south',_'city4') <= 1

In [24]:
for i in regions:
  model += lpSum([x[i,j] for j in cities]) == 1

In [26]:
for j in cities:
  model += lpSum([x[i,j] for i in regions]) <= 4*y[j]

In [27]:
model.solve()

1

In [28]:
LpStatus[model.status]

'Optimal'

In [29]:
vardict = {}
for v in model.variables():
  vardict[v.name] = v.varValue
vardict['z'] = value(model.objective)

In [30]:
vardict

{"x_('east',_'city1')": 0.0,
 "x_('east',_'city2')": 0.0,
 "x_('east',_'city3')": 1.0,
 "x_('east',_'city4')": 0.0,
 "x_('midwest',_'city1')": 0.0,
 "x_('midwest',_'city2')": 0.0,
 "x_('midwest',_'city3')": 1.0,
 "x_('midwest',_'city4')": 0.0,
 "x_('south',_'city1')": 0.0,
 "x_('south',_'city2')": 0.0,
 "x_('south',_'city3')": 1.0,
 "x_('south',_'city4')": 0.0,
 "x_('west',_'city1')": 1.0,
 "x_('west',_'city2')": 0.0,
 "x_('west',_'city3')": 0.0,
 "x_('west',_'city4')": 0.0,
 'y_city1': 1.0,
 'y_city2': 0.0,
 'y_city3': 1.0,
 'y_city4': 0.0,
 'z': 242.0}